In [2]:
#cuilding make_val_step higher function
def make_val_step_fn(model , loss_fn):

    def perform_val_step(x ,y):
        model.eval()  # set model to evals mode 
        yhat = model(x)
        loss = loss_fn(yhat , y)
        #  no need to compute step 3 and 4 , since we dont update parameters during evalution 

        return loss.item()
    
    return perform_val_step



In [3]:
import torch 
import numpy as np 
import torch.nn as nn 
import torch.optim as optim 

In [4]:
%%writefile model_configuration/v2.py
import torch 
import torch.optim as optim 
import torch.nn as nn 
device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 0.01 #set learning rate 
model = nn.Sequential(nn.Linear(1,1).to(device))
optimizer = optim.SGD(model.parameters() , lr = lr)
loss_fn = nn.MSELoss(reduction= 'mean')


def make_train_step_fn(model , loss_fn , optimizer):
    def train_step(x ,y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat , y)
        loss.backward() #calcualte gradients 
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

train_step_fn = make_train_step_fn(model , loss_fn , optimizer)

def make_val_step_fn(model , loss_fn):
    def perform_val_step(x ,y ):
        model.eval() #model evaluation 
        yhat = model(x)
        loss = loss_fn(yhat , y)
        return loss.item()
    return perform_val_step


val_step_fn = make_val_step_fn(model , loss_fn)


Overwriting model_configuration/v2.py


In [10]:
%run model_configuration/v2.py

## now creating model_trannign for our validation data

In [11]:
#train 
import numpy as np 
import torch.nn as nn
import torch.optim as optim 



In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [19]:
%run -i  data_preparation/v2.py
%run -i model_configuration/v2.py

In [23]:
%%writefile model_training/v4.py
def mini_batch(device , train_loader , train_step_fn):
    mini_batch_losses = []
    for x_batch , y_batch in train_loader:
        x_batch = x_batch.to(device).view(-1 , 1)
        y_batch = y_batch.to(device).view(-1 , 1)

        mini_batch_loss = train_step_fn(x_batch , y_batch)
        mini_batch_losses.append(mini_batch_loss)

    loss = np.mean(mini_batch_loss)
    return loss 

        




n_epcohs = 200 
losses = []
val_losses = [] 
for epcoh in range(n_epcohs):
    #inner loop 
    loss = mini_batch(device ,  train_loader , train_step_fn)
    losses.append(loss)

    with torch.no_grad():
        val_loss = mini_batch(device , val_loader , val_step_fn)
        val_losses.append(val_loss)

Writing model_training/v4.py


In [24]:
%run -i model_training/v4.py


In [25]:
model.state_dict()


OrderedDict([('0.weight', tensor([[2.0012]], device='cuda:0')),
             ('0.bias', tensor([9.9912], device='cuda:0'))])

In [26]:
# full pipeline 
%run -i data_preparation/v2.py
%run -i model_configuration/v2.py
%run -i model_training/v4.py
